In [1]:
### importing needed files

import warnings
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

import numpy as np
import pandas as pd
import lightgbm as lgb
import tensorflow as tf
import sklearn as sk
import matplotlib as mpl
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.ensemble import RandomForestClassifier as RF
from xgboost import XGBClassifier
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.calibration import calibration_curve
from sklearn.preprocessing import PolynomialFeatures
from catboost import Pool, CatBoostClassifier
from catboost import CatBoostRegressor
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn import svm, model_selection, tree, linear_model, neighbors, naive_bayes, ensemble
from sklearn import discriminant_analysis, gaussian_process
import os
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifierCV
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_selection import SelectKBest, f_classif
import xgboost as xgb
import numpy as np

# enable the HalvingGridSearchCV
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV


In [2]:
# load CSV files into dataframes with lowercase variable names for x and y
file_path_features = './training_set_features.csv'
file_path_labels = './training_set_labels.csv'

x = pd.read_csv(file_path_features)
y = pd.read_csv(file_path_labels)

In [3]:
# joining the x & y into a single dataframe
z = y.copy()
z.drop('respondent_id', axis=1, inplace=True)
frames = [x, z]
data = pd.concat(frames, axis=1)

In [4]:
columns = data.columns.tolist()
print(columns)

['respondent_id', 'h1n1_concern', 'h1n1_knowledge', 'behavioral_antiviral_meds', 'behavioral_avoidance', 'behavioral_face_mask', 'behavioral_wash_hands', 'behavioral_large_gatherings', 'behavioral_outside_home', 'behavioral_touch_face', 'doctor_recc_h1n1', 'doctor_recc_seasonal', 'chronic_med_condition', 'child_under_6_months', 'health_worker', 'health_insurance', 'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk', 'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective', 'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'age_group', 'education', 'race', 'sex', 'income_poverty', 'marital_status', 'rent_or_own', 'employment_status', 'hhs_geo_region', 'census_msa', 'household_adults', 'household_children', 'employment_industry', 'employment_occupation', 'h1n1_vaccine', 'seasonal_vaccine']


In [5]:
unique_values = data['age_group'].unique()
print(unique_values)


['55 - 64 Years' '35 - 44 Years' '18 - 34 Years' '65+ Years'
 '45 - 54 Years']


In [6]:
def fill_missing_values(column):
    if column.isnull().any():
        if column.dtype != 'object':
            return column.fillna(column.median())
        else:
            return column.fillna(column.mode()[0])
    return column

# applying the function to each column
data = data.apply(fill_missing_values, axis=0)

### Question 2

In [7]:
import pandas as pd

# calculating the correlation matrix
corr_matrix = data[['behavioral_wash_hands', 'behavioral_face_mask', 'behavioral_antiviral_meds', 
                       'behavioral_avoidance', 'h1n1_vaccine', 'seasonal_vaccine']].corr()

# now printing correlation with h1n1 and seasonal vaccines
print("Correlation with H1N1 Vaccine:")
print(corr_matrix['h1n1_vaccine'][['behavioral_wash_hands', 'behavioral_face_mask', 'behavioral_antiviral_meds', 'behavioral_avoidance']])

print("\nCorrelation with Seasonal Flu Vaccine:")
print(corr_matrix['seasonal_vaccine'][['behavioral_wash_hands', 'behavioral_face_mask', 'behavioral_antiviral_meds', 'behavioral_avoidance']])


Correlation with H1N1 Vaccine:
behavioral_wash_hands        0.074570
behavioral_face_mask         0.070413
behavioral_antiviral_meds    0.040226
behavioral_avoidance         0.046837
Name: h1n1_vaccine, dtype: float64

Correlation with Seasonal Flu Vaccine:
behavioral_wash_hands        0.112254
behavioral_face_mask         0.050020
behavioral_antiviral_meds    0.006013
behavioral_avoidance         0.076130
Name: seasonal_vaccine, dtype: float64


In [8]:
# interaction terms
data['wash_mask_interaction'] = data['behavioral_wash_hands'] * data['behavioral_face_mask']
data['wash_antiviral_interaction'] = data['behavioral_wash_hands'] * data['behavioral_antiviral_meds']

X = data[['behavioral_wash_hands', 'behavioral_face_mask', 'behavioral_antiviral_meds', 
             'behavioral_avoidance', 'wash_mask_interaction', 'wash_antiviral_interaction']]


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

data['wash_mask_interaction'] = data['behavioral_wash_hands'] * data['behavioral_face_mask']
data['wash_antiviral_interaction'] = data['behavioral_wash_hands'] * data['behavioral_antiviral_meds']

# these are independent variables including interaction terms
X = data[['behavioral_wash_hands', 'behavioral_face_mask', 'behavioral_antiviral_meds', 
             'behavioral_avoidance', 'wash_mask_interaction', 'wash_antiviral_interaction']]

# dependent variable (H1N1 vaccine)
y_h1n1 = data['h1n1_vaccine']

# dependent variable (Seasonal flu vaccine)
y_seasonal = data['seasonal_vaccine']

X_train_h1n1, X_test_h1n1, y_train_h1n1, y_test_h1n1 = train_test_split(X, y_h1n1, test_size=0.3, random_state=42)
X_train_seasonal, X_test_seasonal, y_train_seasonal, y_test_seasonal = train_test_split(X, y_seasonal, test_size=0.3, random_state=42)

# logistic regression models
model_h1n1 = LogisticRegression(max_iter=1000)
model_seasonal = LogisticRegression(max_iter=1000)

model_h1n1.fit(X_train_h1n1, y_train_h1n1)
model_seasonal.fit(X_train_seasonal, y_train_seasonal)

y_pred_h1n1 = model_h1n1.predict(X_test_h1n1)
y_pred_seasonal = model_seasonal.predict(X_test_seasonal)

print("H1N1 Vaccine Model Evaluation with Interaction Terms")
print(confusion_matrix(y_test_h1n1, y_pred_h1n1))
print(classification_report(y_test_h1n1, y_pred_h1n1))
print("Accuracy:", accuracy_score(y_test_h1n1, y_pred_h1n1))

print("\nSeasonal Flu Vaccine Model Evaluation with Interaction Terms")
print(confusion_matrix(y_test_seasonal, y_pred_seasonal))
print(classification_report(y_test_seasonal, y_pred_seasonal))
print("Accuracy:", accuracy_score(y_test_seasonal, y_pred_seasonal))

# model coefficients to understand the impact of each behavior
print("H1N1 Vaccine Model Coefficients:", model_h1n1.coef_)
print("Seasonal Flu Vaccine Model Coefficients:", model_seasonal.coef_)


H1N1 Vaccine Model Evaluation with Interaction Terms
[[6319    0]
 [1694    0]]
              precision    recall  f1-score   support

           0       0.79      1.00      0.88      6319
           1       0.00      0.00      0.00      1694

    accuracy                           0.79      8013
   macro avg       0.39      0.50      0.44      8013
weighted avg       0.62      0.79      0.70      8013

Accuracy: 0.7885935355048047

Seasonal Flu Vaccine Model Evaluation with Interaction Terms
[[1725 2617]
 [1131 2540]]
              precision    recall  f1-score   support

           0       0.60      0.40      0.48      4342
           1       0.49      0.69      0.58      3671

    accuracy                           0.53      8013
   macro avg       0.55      0.54      0.53      8013
weighted avg       0.55      0.53      0.52      8013

Accuracy: 0.5322600773742668
H1N1 Vaccine Model Coefficients: [[ 0.4400633   0.74395176  0.43405702  0.08802942 -0.24375771 -0.17417697]]
Seasonal F

### Question 1

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from IPython.display import display


# lets convert dependent variables to binary
# 1 = "effective", 0 = "not Effective"
data['binary_h1n1_effectiveness'] = data['opinion_h1n1_vacc_effective'].apply(lambda x: 1 if x >= 4 else 0)
data['binary_h1n1_risk'] = data['opinion_h1n1_risk'].apply(lambda x: 1 if x >= 4 else 0)
data['binary_seasonal_effectiveness'] = data['opinion_seas_vacc_effective'].apply(lambda x: 1 if x >= 4 else 0)
data['binary_seasonal_risk'] = data['opinion_seas_risk'].apply(lambda x: 1 if x >= 4 else 0)

categorical_features = ['age_group', 'education', 'income_poverty'] #encoding

#  column transformer that will apply one-hot encoding to categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features)
    ])

# logistic regression models and pipelines for the binary outcomes
model_h1n1_eff = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('classifier', LogisticRegression(max_iter=1000))])

model_h1n1_risk = Pipeline(steps=[('preprocessor', preprocessor),
                                  ('classifier', LogisticRegression(max_iter=1000))])

model_seas_eff = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('classifier', LogisticRegression(max_iter=1000))])

model_seas_risk = Pipeline(steps=[('preprocessor', preprocessor),
                                  ('classifier', LogisticRegression(max_iter=1000))])

# independent variables (demographic factors)
X = data[['age_group', 'education', 'income_poverty']]

# dependent variables (binary perceptions of vaccine effectiveness and risks)
y_h1n1_effectiveness = data['binary_h1n1_effectiveness']
y_h1n1_risk = data['binary_h1n1_risk']
y_seasonal_effectiveness = data['binary_seasonal_effectiveness']
y_seasonal_risk = data['binary_seasonal_risk']

model_h1n1_eff.fit(X, y_h1n1_effectiveness)
model_h1n1_risk.fit(X, y_h1n1_risk)
model_seas_eff.fit(X, y_seasonal_effectiveness)
model_seas_risk.fit(X, y_seasonal_risk)

# getting the coefficients for the models to understand the impact of each factor
coefficients_h1n1_eff = model_h1n1_eff.named_steps['classifier'].coef_
coefficients_h1n1_risk = model_h1n1_risk.named_steps['classifier'].coef_
coefficients_seas_eff = model_seas_eff.named_steps['classifier'].coef_
coefficients_seas_risk = model_seas_risk.named_steps['classifier'].coef_

# extracting feature names after one-hot encoding
feature_names = model_h1n1_eff.named_steps['preprocessor'].transformers_[0][1].get_feature_names_out(categorical_features)

# combining the coefficients into dataframes
eff_coeffs_h1n1 = pd.DataFrame(coefficients_h1n1_eff, columns=feature_names)
risk_coeffs_h1n1 = pd.DataFrame(coefficients_h1n1_risk, columns=feature_names)
eff_coeffs_seas = pd.DataFrame(coefficients_seas_eff, columns=feature_names)
risk_coeffs_seas = pd.DataFrame(coefficients_seas_risk, columns=feature_names)

# displaying the coefficients in a table format
print("H1N1 Vaccine Effectiveness Model Coefficients:")
display(eff_coeffs_h1n1)

print("\nH1N1 Vaccine Risk Model Coefficients:")
display(risk_coeffs_h1n1)

print("\nSeasonal Flu Vaccine Effectiveness Model Coefficients:")
display(eff_coeffs_seas)

print("\nSeasonal Flu Vaccine Risk Model Coefficients:")
display(risk_coeffs_seas)


H1N1 Vaccine Effectiveness Model Coefficients:


,age_group_18 - 34 Years,age_group_35 - 44 Years,age_group_45 - 54 Years,age_group_55 - 64 Years,age_group_65+ Years,education_12 Years,education_< 12 Years,education_College Graduate,education_Some College,"income_poverty_<= $75,000, Above Poverty","income_poverty_> $75,000",income_poverty_Below Poverty
0,0.247422,0.083541,0.081367,0.122413,-0.005328,0.133523,-0.017701,0.269278,0.144314,0.048851,0.418512,0.062051



H1N1 Vaccine Risk Model Coefficients:


,age_group_18 - 34 Years,age_group_35 - 44 Years,age_group_45 - 54 Years,age_group_55 - 64 Years,age_group_65+ Years,education_12 Years,education_< 12 Years,education_College Graduate,education_Some College,"income_poverty_<= $75,000, Above Poverty","income_poverty_> $75,000",income_poverty_Below Poverty
0,-0.028717,0.109059,-0.062101,-0.113067,-0.361747,-0.158388,0.116832,-0.224115,-0.190903,-0.368154,-0.179947,0.091527



Seasonal Flu Vaccine Effectiveness Model Coefficients:


,age_group_18 - 34 Years,age_group_35 - 44 Years,age_group_45 - 54 Years,age_group_55 - 64 Years,age_group_65+ Years,education_12 Years,education_< 12 Years,education_College Graduate,education_Some College,"income_poverty_<= $75,000, Above Poverty","income_poverty_> $75,000",income_poverty_Below Poverty
0,0.053597,-0.023344,0.026904,0.287686,0.483608,0.232674,0.064788,0.330223,0.200766,0.244556,0.53422,0.049676



Seasonal Flu Vaccine Risk Model Coefficients:


,age_group_18 - 34 Years,age_group_35 - 44 Years,age_group_45 - 54 Years,age_group_55 - 64 Years,age_group_65+ Years,education_12 Years,education_< 12 Years,education_College Graduate,education_Some College,"income_poverty_<= $75,000, Above Poverty","income_poverty_> $75,000",income_poverty_Below Poverty
0,-0.129029,-0.073439,-0.094083,0.062335,0.057103,-0.046202,0.058317,-0.115457,-0.073772,-0.226313,-0.019067,0.068267
